# 🎬 Netflix Movies and TV Shows — Data Cleaning & Preprocessing

This notebook serves as **Phase 1** of the Netflix EDA Project.  
Our goal in this phase is to clean and prepare the dataset for exploration by:
- Fixing encoding issues  
- Handling missing values  
- Standardizing data formats  
- Extracting useful columns (like release year and duration)  

We'll end up with a clean dataset that’s analysis-ready for the next phase.

In [4]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np

# Step 2: Load the dataset
df = pd.read_csv("netflix_titles.csv")

# Step 3: Basic info
print(f"Shape of dataset: {df.shape}")
df.info()

# Step 4: Quick look at data
df.head()

ModuleNotFoundError: No module named 'pandas'

In [2]:
!python3 -m pip install pandas numpy matplotlib seaborn plotly

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.8 MB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 9.8 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 8.3 MB/s eta 0:00:01
     |████████████████████████████████| 229 kB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 249 kB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 8.7 MB/s eta 0:00:011
     |████████████████████████████████| 4.7 MB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 113 kB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 7.0 MB/s eta 0:00:01
     |███████████████████████████████

In [6]:
import sys
print(sys.executable)


/opt/homebrew/opt/python@3.10/bin/python3.10


In [7]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install pandas numpy matplotlib seaborn plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 5.8 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 6.3 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 6.0 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 5.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 6.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 6.6 MB/s  0:00:00 eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [seaborn]2/14 [matplotlib]


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

print("✅ All libraries successfully imported!")

Matplotlib is building the font cache; this may take a moment.


✅ All libraries successfully imported!
